# Migration WorkFlow Example

This is a more complex scenario that describes a workflow where there are two service engine groups that need to be merged into a single one. We will define a **TARGET** service engine group that will be used to receive the configuration extracted from the **SOURCE** service engine group. 

<img src="images/migration_scenario.png">

- [1.- Initial Login](#1--initial-login)
- [2.- Source Service Engine Data Collection](#step-1--collecting-source-service-engine-group-data)


- STEP 1.- Read Source SE Group information

- STEP 2.- Select interface to be migrated from source SE

- STEP 3.- Creating Cloned objects at target SE Group 
   - VRF_NEW
   - NETWORK_NEW
   - NETWORK_SERVICE_NEW
   - INTERFACE_CONFIG 
   - VSVIPs_NEW
   - POOLS_NEW
   - VIRTUAL_SERVICES_NEW (DISABLED STATE)

- STEP 4.- Proceed with Migration 
   - 1 Disabling interfaces at Source SE 
   - 2 Disabling VS at Source
   - 3 Enabling VS at TARGET SEG

- STEP 5.- Verification
          - Possible Roll-Back (optional)

- STEP 6.- Cleaning old configurations and renaming to original names
        - Delete migrated VSs
        - Delete migrated VSVIPs
        - Delete migrated Network Service
        - Delete migrated Network
        - Delete migrated VRF
        - Delete migrated Service Engines
        - Delete migrated Service Engine Group
        - Restoring original names 

In [17]:
from envs.controller_info import session_params as session_env
### Import python packages to setup Demo environment.
#from libs.migration_scenario_load import setup_scenario
#from libs.migration_scenario_clean import clean_scenario

In [ ]:
# Clean Scenario
#clean_scenario(session_env)
#setup_scenario(session_env)   

## Initial Login

In [18]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import pandas as pd
from IPython.display import display, Image
from libs.aux import *

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:ynp2duyuvbo9p788m96vkselruh43lqb


### Select  SOURCE and TARGET Service Engine Group Names using Interactive Menu

In [19]:
# Get SOURCE AND TARGET Service Engine Group Names from the list using menu
query = {
    "fields": "name"
}

resp = api.get("serviceenginegroup", params=query)
resp = json.loads(resp.text)["results"]

menu_options = []
i = 0
for item in resp:
    menu_options.insert(i, "Service Engine Group: "+item["name"])
    i = i +1

menu_title = "Please Select an option for SOURCE Service Engine Group to be migrated:"
selected_option = display_menu_from_list(menu_options, menu_title)
    
if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as SOURCE")
else:
    print("No valid option was selected.")

source_segroup = selected_option.split(": ")[1]

print()
menu_title = "Please Select an option for TARGET Service Engine Group:"
selected_option = display_menu_from_list(menu_options, menu_title)

if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as TARGET")
    if selected_option.split(": ")[1] == source_segroup:
        while selected_option.split(": ")[1] == source_segroup:
          print()
          print (" !!!!! SOURCE AND TARGET Service Engine Groups cannot be the same !!!!! Repeat Selection for TARGET...")
          menu_title = "Please Select an different option for TARGET Service Engine Group:"
          selected_option = display_menu_from_list(menu_options, menu_title)
          print(selected_option)
else:
    print("No valid option was selected.")

target_segroup = selected_option.split(": ")[1] 

Please Select an option for SOURCE Service Engine Group to be migrated:
--------------------------------------------------
1. Service Engine Group: Default-Group
2. Service Engine Group: SEG-SOURCE-MAD-002
3. Service Engine Group: SEG-TARGET-MAD-001
You selected: Service Engine Group: SEG-SOURCE-MAD-002 as SOURCE

Please Select an option for TARGET Service Engine Group:
--------------------------------------------------
1. Service Engine Group: Default-Group
2. Service Engine Group: SEG-SOURCE-MAD-002
3. Service Engine Group: SEG-TARGET-MAD-001
You selected: Service Engine Group: SEG-TARGET-MAD-001 as TARGET


In [ ]:
# Set SOURCE AND TARGET Service Engine Group Names from the list above
# Source = Group to be migrated
#source_segroup = "SEG-SOURCE-MAD-002"
#target_segroup = "SEG-TARGET-MAD-001"

## STEP 1.- Collecting Source Service Engine Group Data

The first step is to Gather SE Group Information
- Service Engine Group Configuration
- Service Engine Configuration (VRF included)
- Network Configuration (IP Routes)
- Virtual Services Configuration
- Network Services Configuration (if exists)

In [31]:
# Gather SOURCE Service Engine Group related configuration
print("\033[1mExtracting SOURCE Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

# Define GET request parameters
url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": source_segroup,
   "include_name": "true"
}
# Send GET Request
resp = api.get(url_path, params=query)
#print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   #print(resp)
   #print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_segroup_data = resp_data[0]

# Extract related Service Engine
print()
print("\033[1mExtracting SOURCE Services Engines information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

# GET Service Engine Related Configuration
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}",
   "include_name": "true"
}

# Send GET Request
resp = api.get(url_path, params=query)
#print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   #print(resp)
   #print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_se_data = resp_data


Extracting SOURCE Service Engine Group Information
--------------------------------------------------
The following serviceenginegroup names has been found:
['SEG-SOURCE-MAD-002']


Extracting SOURCE Services Engines information
--------------------------------------------------
The following serviceengine names has been found:
['192.168.1.22', '192.168.1.23']



In [ ]:

# Extract related virtual services
# GET Virtual Services Related Configuration
print("\033[1mExtracting SOURCE Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

seg_data = source_segroup_data
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}",
   "include_name": "true"
}

# Send GET Request
resp = api.get(url_path, params=query)
#print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   #print(resp)
   #print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_vs_data = resp_data

# Extract
# Extract Placement  related Service Engine
print("\033[1mExtracting SOURCE Placement Networks \033[0m")
print("\033[1m------------------------------------\033[0m")

source_networks = []
for i in range(len(source_vs_data)):
   vsvip_name = source_vs_data[i]["vsvip_ref"].split("#")[1]
   vsvip = api.get_object_by_name("vsvip", vsvip_name)
   vsvip["vip"][0]["placement_networks"][0]["network_ref"]
   vsvip_uuid = vsvip["vip"][0]["placement_networks"][0]["network_ref"].split("/")[5]
   query = {
      "uuid": vsvip_uuid,
      "include_name": "true"
   }
   resp = api.get("network", params=query)
   network_name = json.loads(resp.text)["results"][0]["name"]
   if network_name not in source_networks:
       source_networks.append(network_name)
print ("The following placement networks has been found:")
print(source_networks)

# Extract related Service Engine
print()
print("\033[1mExtracting SOURCE Services Engines information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

# GET Service Engine Related Configuration
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}",
   "include_name": "true"
}

# Send GET Request
resp = api.get(url_path, params=query)
#print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   #print(resp)
   #print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_se_data = resp_data

## Exploring Source Information

In [32]:
# Exploring Interface Acapters Information of SOURCE SE
source_if_name=[]
for index in range(len(source_se_data)):
    print("\033[1mShowing information of the SOURCE Service Engine "+str(index+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = source_se_data[index]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    menu_options=[]
    for i in range(len(se_data_vnics)):
        if "vnic_networks" in se_data_vnics[i]:
            if_name = se_data_vnics[i]["if_name"]
            vrf_name = se_data_vnics[i]["vrf_ref"].split("#")[1]
            mac_address = se_data_vnics[i]["mac_address"]
            ip_addr = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["addr"]
            type = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["type"]
            mask = se_data_vnics[i]["vnic_networks"][0]["ip"]["mask"]
            mode = se_data_vnics[i]["vnic_networks"][0]["mode"]
            menu_options = ["Interface "+ if_name+" at VRF "+vrf_name+" with mac "+ mac_address+" and IP"+type+" ADDRESS "+ip_addr+"/"+str(mask)]
            menu_title = "Please Select a candidate interface for SOURCE Service Engine "+str(index+1)+" from above list to be migrated:"
            print()
            selected_option = display_menu_from_list(menu_options, menu_title)
            if selected_option:
                selected_if_name = selected_option.split(" ")[1]
                print("You selected: \033[1m"+selected_if_name+"\033[0m as SOURCE for Service Engine "+str(index+1))
                print()
                #if selected_if_name not in source_if_name:
                source_if_name.insert(index,selected_if_name)
            else:
                print("No valid option was selected.")


Showing information of the SOURCE Service Engine 1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:83:62,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:83:6c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:83:76,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:83:80,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.2.22', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:83:94,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:83:9e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:83:a8,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:83:b2,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for SOURCE Service Engine 1 from above list to be migrated:
--------------------------------------------------
1. Interface eth1 at VRF vrf_source_mad_002 with mac 00:0c:29:a7:83:8a and IPV4 ADDRESS 192.168.2.22/24
You selected: eth1 as SOURCE for Service Engine 1

Showing information of the SOURCE Service Engine 2 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:61:16,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:61:20,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:61:2a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:61:34,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.2.23', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:61:48,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:61:52,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:61:5c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:61:66,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for SOURCE Service Engine 2 from above list to be migrated:
--------------------------------------------------
1. Interface eth1 at VRF vrf_source_mad_002 with mac 00:0c:29:a7:61:3e and IPV4 ADDRESS 192.168.2.23/24
You selected: eth1 as SOURCE for Service Engine 2



In [22]:
# Variable initialization
# A variable named output_config will be used to store extracted information in a single dictionary object
output_config={}

# Extract Interface Information from Source Service Engines
for i in range(len(source_se_data)):
    config=[]
    print("\033[1mExtracting information from SOURCE SE "+str(i+1)+" interface to migrate "+source_if_name[i]+" \033[0m")
    print("\033[1m----------------------------------------------------------------------------------------------------\033[0m")
    se_data_vnics = source_se_data[i]["data_vnics"]
    se_data_vnic_to_migrate = [ adapter for adapter in se_data_vnics if adapter.get("if_name") == source_if_name[i]]
    #se_data_vnic_to_migrate_df = pd.DataFrame(se_data_vnic_to_migrate)
    time.sleep(1/4)
    #display(se_data_vnic_to_migrate_df)
    se_vnic_to_migrate_config = se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]
    se_vnic_to_migrate_config_df = pd.DataFrame(se_vnic_to_migrate_config)

    # Extract VRF UUID from the vrf_ref
    
    vrf_ref = se_data_vnic_to_migrate[0]['vrf_ref']
    vrf_uuid = vrf_ref.split("/")[5].split("#")[0]
    vrf_name = vrf_ref.split("#")[1]
    
    vrf_config = {  "vrfcontext": vrf_name,
                    "vrf_ref": vrf_ref,
                    "vrf_uuid": vrf_uuid
             }

    # Insert information into output_config dictionary
    output_config["ip_routing_config"] = vrf_config

    # Writing VNIC extracted variables 
    se_uuid = source_se_data[i]["uuid"]
    ip_addr = se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]["ip_addr"]["addr"]
    mask = se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]["mask"]
    mac_address = se_data_vnic_to_migrate[0]["mac_address"]
    if_name = source_if_name[i]
    vrf_ref = se_data_vnic_to_migrate[0]["vrf_ref"]

    # Printing Interface configuration
    print("   - Found IP Address "+ip_addr+"/"+str(mask))
    print("   - Found MAC Address "+mac_address)
    print("   - Found VRF Context "+vrf_name)
    print()
  
  
  # Populate a dictionary with extraced data
    config= {
              "se_uuid": se_uuid,
              "ip_addr":ip_addr,
              "mask": mask,
              "mac_address": mac_address,
              "if_name": source_if_name[i],
              "vrf_ref": vrf_ref
              }
  # Populate a new key SE_n with discovered information for that particular interface
    output_config["SE_"+str(i+1)] = config

print("\033[1mExtracting IP Routing Information for the VRF Context " +vrf_name+ " where "+source_if_name[i]+" is attached to \033[0m")
print("\033[1m----------------------------------------------------------------------------------------------------\033[0m")

resp = api.get_object_by_name("vrfcontext", vrf_name)
# Extracting IP Static Route information
try:
  static_routes =[]
  for i in range(len(resp["static_routes"])):
    route_type = resp["static_routes"][i]["prefix"]["ip_addr"]["type"]
    route_prefix = resp["static_routes"][i]["prefix"]["ip_addr"]["addr"]
    route_mask = resp["static_routes"][i]["prefix"]["mask"]
    route_next_hop = resp["static_routes"][i]["next_hop"]["addr"]
    print("   - Found static IP"+route_type+" route prefix "+route_prefix+"/"+str(route_mask)+" pointing to next-hop --> "+route_next_hop)
    static_routes.insert(i, {
        "route_type": route_type,
        "route_prefix": route_prefix,
        "route_mask": route_mask,
        "route_next_hop": route_next_hop
    })
except:
    print("   - No Static routes found")
print()

# Add Extracted routing information to the dictionary
output_config["ip_routing_config"]["static_routes"]=static_routes
print("\033[1mExtracting Network Service Information for the VRF Context "+vrf_name+" and Service Engine Group "+source_segroup+" \033[0m")
print("\033[1m----------------------------------------------------------------------------------------------------\033[0m")

# Extract SE Group REF from SEGROUP Info (url corresponds to se_group_ref in referring objects) 
se_group_uuid = source_segroup_data["url"].split("/")[5].split("#")[0]


# Read Network Service attached to the VRF and Source SEGroup 
query = {
  "refers_to": "vrfcontext:"+vrf_uuid,
  "search": "(se_group_ref,"+se_group_uuid+")"
}
resp = api.get('networkservice', params=query)
if resp.status_code in range(200, 299):
   # Convert response JSON into Python Dictionary
   if json.loads(resp.text)["count"] == 0:
    print("No Network Service Found for that particular VRF/SE_GROUP")
   else:
    network_service_config = json.loads(resp.text)["results"][0]
    print("Found Network Service for that VRF/SE_GROUP named "+network_service_config["name"])
    output_config["network_service_config"]=network_service_config
    try:
        for i in range(len(network_service_config["routing_service"]["floating_intf_ip"])):
            float_addr = network_service_config["routing_service"]["floating_intf_ip"][i]["addr"]
            float_type = network_service_config["routing_service"]["floating_intf_ip"][i]["type"]
            print("   - Found floating IP"+float_type+" "+float_addr)
    except:
            print("   - No Floating IP Addresses found")
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)


Extracting information from SOURCE SE 1 interface to migrate eth1 
----------------------------------------------------------------------------------------------------
   - Found IP Address 192.168.2.22/24
   - Found MAC Address 00:0c:29:a7:83:8a
   - Found VRF Context vrf_source_mad_002

Extracting information from SOURCE SE 2 interface to migrate eth1 
----------------------------------------------------------------------------------------------------
   - Found IP Address 192.168.2.23/24
   - Found MAC Address 00:0c:29:a7:61:3e
   - Found VRF Context vrf_source_mad_002

Extracting IP Routing Information for the VRF Context vrf_source_mad_002 where eth1 is attached to 
----------------------------------------------------------------------------------------------------
   - Found static IPV4 route prefix 192.168.0.0/16 pointing to next-hop --> 192.168.2.10
   - Found static IPV4 route prefix 0.0.0.0/0 pointing to next-hop --> 192.168.2.1

Extracting Network Service Information for the

In [38]:
output_config

{'ip_routing_config': {'vrfcontext': 'vrf_source_mad_002',
  'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2#vrf_source_mad_002',
  'vrf_uuid': 'vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2',
  'static_routes': [{'route_type': 'V4',
    'route_prefix': '192.168.0.0',
    'route_mask': 16,
    'route_next_hop': '192.168.2.10'},
   {'route_type': 'V4',
    'route_prefix': '0.0.0.0',
    'route_mask': 0,
    'route_next_hop': '192.168.2.1'}]},
 'SE_1': {'se_uuid': 'se-564d63c6-3882-ca52-7be5-b04c9ba78358',
  'ip_addr': '192.168.2.22',
  'mask': 24,
  'mac_address': '00:0c:29:a7:83:8a',
  'if_name': 'eth1',
  'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2#vrf_source_mad_002'},
 'SE_2': {'se_uuid': 'se-564d0f61-3a27-ff26-8af7-306296a7610c',
  'ip_addr': '192.168.2.23',
  'mask': 24,
  'mac_address': '00:0c:29:a7:61:3e',
  'if_name': 'eth1',
  'vrf_ref': 'https://192.168.1.15/api/vrfcontext/vr

In [46]:

# Extract related virtual services
# GET Virtual Services Related Configuration
print("\033[1mExtracting SOURCE Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

#seg_data = source_segroup_data
# Define GET request parameters
se_group_uuid=source_segroup_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": "serviceenginegroup:"+se_group_uuid,
   "include_name": "true",
   "search": "(vrf_context_ref,"+vrf_uuid+")",
   "fields": "se_group_ref,vrf_context_ref,vsvip_ref,pool_ref"
}

# Send GET Request
resp = api.get(url_path, params=query)
#print(json.dumps(json.loads(resp.text), indent=3))
# Control Response Status Code
if resp.status_code in range(200, 299):
   #print(resp)
   #print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)
   

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_vs_data = resp_data
output_config["virtualservices"] = source_vs_data


Extracting SOURCE Virtual Services Information
--------------------------------------------------
The following virtualservice names has been found:
['VS_SOURCE_151', 'VS_SOURCE_152']



In [47]:

# Extract
# Extract Placement Network Configuration
print("\033[1mExtracting SOURCE Placement Networks \033[0m")
print("\033[1m------------------------------------\033[0m")

source_networks = []
for i in range(len(source_vs_data)):
   vsvip_name = source_vs_data[i]["vsvip_ref"].split("#")[1]
   vsvip = api.get_object_by_name("vsvip", vsvip_name)
   vsvip["vip"][0]["placement_networks"][0]["network_ref"]
   vsvip_uuid = vsvip["vip"][0]["placement_networks"][0]["network_ref"].split("/")[5]
   query = {
      "uuid": vsvip_uuid,
      "include_name": "true"
   }
   resp = api.get("network", params=query)
   network_name = json.loads(resp.text)["results"][0]["name"]
   if network_name not in source_networks:
       source_networks.append(network_name)
print ("The following placement networks has been found:")
print(source_networks)


#else:
#    print("No Network Service Found for that particular VRF/SE_GROUP")

Extracting SOURCE Placement Networks 
------------------------------------
The following placement networks has been found:
['network_source_mad_002']


In [48]:
a = []


In [49]:
b = {"a":2, "c": 3}

In [51]:
a["vsvip"].insert(0, b)

TypeError: list indices must be integers or slices, not str

In [23]:
print (" Showing full extracted config...")
print(json.dumps(output_config, indent=3, sort_keys=True))

 Showing full extracted config...
{
   "SE_1": {
      "if_name": "eth1",
      "ip_addr": "192.168.2.22",
      "mac_address": "00:0c:29:a7:83:8a",
      "mask": 24,
      "se_uuid": "se-564d63c6-3882-ca52-7be5-b04c9ba78358",
      "vrf_ref": "https://192.168.1.15/api/vrfcontext/vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2#vrf_source_mad_002"
   },
   "SE_2": {
      "if_name": "eth1",
      "ip_addr": "192.168.2.23",
      "mac_address": "00:0c:29:a7:61:3e",
      "mask": 24,
      "se_uuid": "se-564d0f61-3a27-ff26-8af7-306296a7610c",
      "vrf_ref": "https://192.168.1.15/api/vrfcontext/vrfcontext-054e4876-bf3a-4124-ab94-ce379f1a11f2#vrf_source_mad_002"
   },
   "ip_routing_config": {
      "static_routes": [
         {
            "route_mask": 16,
            "route_next_hop": "192.168.2.10",
            "route_prefix": "192.168.0.0",
            "route_type": "V4"
         },
         {
            "route_mask": 0,
            "route_next_hop": "192.168.2.1",
            "rout

In [ ]:
## Cloning Objects 

In [25]:
# Import Cloning Libraries 
from libs.clone import *

# 1 Clone existing VRF object
# Returns a new dictionary with cloned object 
source_vrf = output_config["ip_routing_config"]["vrfcontext"]
cloned_vrf = clone_vrf (session_env, source_vrf)
print()
print("\033[1mCheck returned output for further verification:\033[0m")
print("\033[1m-----------------------------------------------\033[0m")
print(json.dumps(cloned_vrf, indent=3, sort_keys=True))

Cloning existing vrfcontext vrf_source_mad_002 into vrf_source_mad_002-NEW
<Response [201]>
- New vrfcontext named vrf_source_mad_002-NEW CREATED

Check returned output for further verification:
-----------------------------------------------
{
   "_last_modified": "1724926585891650",
   "cloud_ref": "https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1#Default-Cloud",
   "debugvrfcontext": {
      "command_buffer_interval": 2,
      "command_buffer_size": 32768
   },
   "internal_gateway_monitor": {
      "disable_gateway_monitor": false,
      "gateway_monitor_failure_threshold": 10,
      "gateway_monitor_interval": 1000,
      "gateway_monitor_success_threshold": 15
   },
   "lldp_enable": true,
   "name": "vrf_source_mad_002-NEW",
   "static_routes": [
      {
         "next_hop": {
            "addr": "192.168.2.10",
            "type": "V4"
         },
         "prefix": {
            "ip_addr": {
               "addr": "192.168.0.0",
               "type": 

In [29]:
source_networks = ["s"]
for item in source_networks:
    print(item)

network_source_mad_002


In [26]:
# 2 Clone network object in the previously cloned VRF
# Returns a new dictionary with cloned object 
target_vrf = cloned_vrf["name"]
cloned_networks = []
i = 0 
for source_network in source_networks:
  cloned_network = clone_network (session_env, source_network, target_vrf)
  cloned_networks = cloned_networks.insert(i, cloned_network)
  i = i +1
print()
print("\033[1mCheck returned output for further verification:\033[0m")
print("\033[1m-----------------------------------------------\033[0m")
print(json.dumps(cloned_networks, indent=3, sort_keys=True))

Cloning existing network network_source_mad_002 into network_source_mad_002-NEW at VRF vrf_source_mad_002-NEW
<Response [201]>
- New network named network_source_mad_002-NEW CREATED

Check returned output for further verification:
-----------------------------------------------
null


In [28]:
cloned_networks

In [16]:

b = {"a": 2, "b":2}
i = 0
a.insert(1, b)
a


[{'a': 2, 'b': 2},
 {'a': 2, 'b': 2},
 {'a': 2, 'b': 2},
 {'a': 2, 'b': 2},
 {'a': 2, 'b': 2}]

In [12]:
a

[{'a': 2, 'b': 2}]

In [ ]:
type(a)

In [ ]:
cloned_networks = cloned_networks.insert(0, cloned_network)

### TARGET SERVICE ENGINE GROUP

In [ ]:
# GET Service Engine Group Configuration

# Define GET request parameters
# Extract related Service Engine
print("\033[1mExtracting TARGET Services Engine Group information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": target_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)

# Control Response Status Code
if resp.status_code in range(200, 299):
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_segroup_data = resp_data[0]

# Extract related virtual services
# GET Virtual Services Related Configuration
print("\033[1mExtracting TARGET Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

seg_data = target_segroup_data
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)

# Control Response Status Code
if resp.status_code in range(200, 299):
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_vs_data = resp_data

# Extract related Service Engine

# GET Service Engine Related Configuration
print("\033[1mExtracting TARGET Services Engine Group information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")

segroup_data = target_segroup_data
# Define GET request parameters
se_group_uuid=segroup_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)

# Control Response Status Code
if resp.status_code in range(200, 299):
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)
 
# Save Result
target_se_data = resp_data

In [ ]:
# Exploring Interface Information of SOURCE SE

for i in range(len(target_se_data)):
    print("\033[1mShowing information of the TARGET SE "+str(i+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = target_se_data[i]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    time.sleep(1/4)

### Select one FREE interface at Target Service Engines that interface that will be used to receive gathered ip address configuration from source interface at source Service Engines


In [ ]:
target_if_name = "eth2"

In [ ]:
# Assuming IP Address will be REUSED!!!
#  1.- Migrate VS (change SEGroup)
#  2.- Migrate SEs interfaces
#  3.- Migrate Network Services (if exists)
# Disable MIGRATING SE INTERFACES TO AVOID ITF CLASHING

## 2.1  Migrate Service Engine Interfaces

In [ ]:
# Display gathered information prior to execute next cell


print ("The following interfaces has been selected for migration")
print()
for i in range(len(output_config)):
    print("For the SOURCE Service Engine "+str(i+1)+" with name \033[1m" + source_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + source_if_name + "\033[0m has been selected with following configuration")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
for i in range(len(target_se_data)):
    print("For the TARGET Service Engine "+str(i+1)+" with name \033[1m" + target_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + target_if_name + "\033[0m will be configured with above information.")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
print("WARNING!!! Before proceeding migrated interfaces in the source SEs will be disabled")

In [ ]:
# Create dictionary list containing pair of migrating-target SEs  
se_pairs =[
     {"source_se_uuid": source_se_data[0]["uuid"],
      "target_se_uuid": target_se_data[0]["uuid"]
     },
     {"source_se_uuid": source_se_data[1]["uuid"],
     "target_se_uuid": target_se_data[1]["uuid"]
     }
]
# Loop to migrate 2 se Pairs (migrating to target) 
for i in range(len(se_pairs)):
    source_se_uuid = se_pairs[i]["source_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]
    print("Migrating pair "+str(i+1))
    print("-------------------------")
    print("Migrating (source) Service Engine UUID \033[1m"+source_se_uuid +"\033[0m at SE Group \033[1m"+source_segroup+"\033[0m") 
    print("Receiving (target) Service Engine UUID \033[1m"+target_se_uuid+"\033[0m at SE Group \033[1m"+target_segroup+"\033[0m")
    source_se_uuid = se_pairs[i]["source_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]

    # Disabling interface to migrate at source SE

    # Extract the source/target SE data with matching uuid
    source_se_json_data = [ se for se in source_se_data if se.get("uuid") == source_se_uuid]
    target_se_json_data = [ se for se in target_se_data if se.get("uuid") == target_se_uuid]


    # Extract source_interface vnic_networks object
    json_data_source_adapter = [ data_vnic for data_vnic in source_se_json_data[0]["data_vnics"] if data_vnic.get("if_name") == source_if_name ]
    json_data_source_adapter_vnic_networks = [json_data_source_adapter[0]["vnic_networks"][0]]

    # Modifying existing value for selected interface at source SE (i.e Enabled = False for corresponding interface to shutdown )
    # vnic_network list must be removed to avoid SE interface overlapping
    if_name_key = "if_name"
    if_name_value = source_if_name

    # Loop through each dictionary in the list
    for adapter in source_se_json_data[0]["data_vnics"]:
        # Check if the key_to_check matches the value_to_match
        if adapter.get(if_name_key) == if_name_value:
            # Update the value of key_to_modify
            adapter["enabled"] = False
            adapter["vnic_networks"]=[]
    
    # Remove _last_modified key
    source_se_json_data[0].pop("_last_modified", None)

    # Disable migrating interfaces to avoid IP address overlapping
    body = source_se_json_data[0]
    print ("Applying changes to source SE "+str(i+1)+" with uuid "+ source_se_uuid)
    print (" - Changing interface "+source_if_name+" to disabled state")
    print (" - Removing IP Address Configuration")

    url_path = "serviceengine/"+source_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

    # PUT to configure migrating IP Addresses in target SE
    # Extract IP Address from migrating SE
    # Importing extracted information into target body
    if_name_key = "if_name"
    if_name_value = target_if_name

    # Loop through each dictionary in the list
    for item in target_se_json_data[0]["data_vnics"]:
      # Check if the interface name correspondes to given value
      if item.get(if_name_key) == if_name_value:
        # Update the value of vnic_networks from source SE
        item["vnic_networks"] = json_data_source_adapter_vnic_networks
    
    # Remove _last_modified key
    target_se_json_data[0].pop("_last_modified", None)

    body = target_se_json_data[0]
    print ("Applying changes to target SE "+str(i+1)+" with uuid "+ target_se_uuid)
    print (" - Applying IP Address Configuration")
    print(target_se_json_data[0])

    url_path = "serviceengine/"+target_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

## 2.1  Migrate Virtual Services

In [ ]:
# Increase the number of VS per SE to acommodate imported VS
target_max_vs_per_se = target_segroup_data["max_vs_per_se"]
count_of_source_imported_vs = len(source_vs_data)
count_of_current_target_vs = len(target_vs_data)
if ((count_of_source_imported_vs + count_of_current_target_vs) >= (target_max_vs_per_se - 5)):
    target_segroup_data["max_vs_per_se"] = count_of_source_imported_vs + count_of_current_target_vs + 5
    print ("Increase the number of VS per SE to "+str(target_segroup_data["max_vs_per_se"])+" to acommodate imported VSs ")
    body = target_segroup_data
    # Remove _last_modified key to avoid concurrent update error
    body.pop("_last_modified", None)

    url_path = "serviceenginegroup/"+target_segroup_data["uuid"]
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
       print(resp)
       print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
       print()
    else:
       print('Error in modifying '+url_path+' :%s' % resp.text)

In [ ]:
# Obtain the target se_group_ref (corresponds to key url of the serviceenginegroup object ) 
target_se_group_ref = target_segroup_data["url"]

for item in source_vs_data:
    item["se_group_ref"]=target_se_group_ref
    print ("Service engine group reference has been changed for Virtual Service "+ item["name"])
    print ("Applying new configuration")

    # Remove _last_modified key to avoide concurrent update error
    item.pop("_last_modified", None)

    url_path = "virtualservice/"+item["uuid"]
    resp = api.put (url_path, data=json.dumps(item))

    if resp.status_code in range(200, 299):
       print(resp)
       print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
       print()
    else:
       print('Error in modifying '+url_path+' :%s' % resp.text)